In [1]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

In [2]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [3]:
experiments = client.search_experiments()
for exp in experiments:
    print(exp)

<Experiment: artifact_location='/workspaces/mlops_zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1730173427741, experiment_id='1', last_update_time=1730173427741, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>
<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1730172581211, experiment_id='0', last_update_time=1730172581211, lifecycle_stage='active', name='Default', tags={}>


In [4]:
client.create_experiment(name="my-cool-experiment")

'2'

In [6]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [7]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: cac7a697e69b4cf3b9ccb63358d9a23f, rmse: 5.1395
run id: 906fc591a38a40269d9fe2d0555c06a6, rmse: 5.1395
run id: d0f64c62c3b64727b255cf4ac3b1067b, rmse: 5.1795
run id: 31fd45870c9f4d17bddab5dcc0b95626, rmse: 5.1920
run id: e48ea02efa2b4dbb99281d3d1f91644a, rmse: 5.1949


In [8]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [9]:
run_id = "cac7a697e69b4cf3b9ccb63358d9a23f"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1730175398658, current_stage='None', description=None, last_updated_timestamp=1730175398658, name='nyc-taxi-regressor', run_id='cac7a697e69b4cf3b9ccb63358d9a23f', run_link=None, source='/workspaces/mlops_zoomcamp/02-experiment-tracking/mlruns/1/cac7a697e69b4cf3b9ccb63358d9a23f/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [16]:
model_name = "nyc-taxi-regressor"
# Print all latest versions for debugging
latest_versions = client.get_latest_versions(model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Production
version: 2, stage: Staging
version: 3, stage: None


/tmp/ipykernel_12800/2124872538.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(model_name)


In [14]:
model_version = 2
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/tmp/ipykernel_12800/4090915774.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1730175230655, current_stage='Staging', description='', last_updated_timestamp=1730175475289, name='nyc-taxi-regressor', run_id='028a64dd2647428da71a42c276d25fac', run_link='', source='/workspaces/mlops_zoomcamp/02-experiment-tracking/mlruns/1/028a64dd2647428da71a42c276d25fac/artifacts/model', status='READY', status_message=None, tags={'model': 'random_forest_regressor'}, user_id=None, version=2>

In [15]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1730175230655, current_stage='Staging', description='The model version 2 was transitioned to Staging on 2024-10-29', last_updated_timestamp=1730175476580, name='nyc-taxi-regressor', run_id='028a64dd2647428da71a42c276d25fac', run_link='', source='/workspaces/mlops_zoomcamp/02-experiment-tracking/mlruns/1/028a64dd2647428da71a42c276d25fac/artifacts/model', status='READY', status_message=None, tags={'model': 'random_forest_regressor'}, user_id=None, version=2>